In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix

In [3]:
df = pd.read_csv("OASIS4_data_clinical.csv")

In [4]:
%matplotlib inline

import matplotlib.pyplot as plt

# Your plotting code here

plt.show()  # Display the plot in the Jupyter Notebook output


In [6]:
# Import the necessary module
from ydata_profiling import ProfileReport

# Create a ProfileReport object
report = ProfileReport(df)

# Generate the data profiling report
report.to_file('oasis_4_2.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
df.dropna(inplace=True)

In [11]:
df.drop('oasis_id' , axis=1,inplace=True)
df.drop('demographics_id', axis=1,inplace=True)
df.drop('demographics_firstvisit', axis=1,inplace=True)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 660 entries, 0 to 662
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   sex                   660 non-null    int64  
 1   age                   660 non-null    int64  
 2   edu                   660 non-null    int64  
 3   race                  660 non-null    int64  
 4   hispanic              660 non-null    int64  
 5   marriage              660 non-null    int64  
 6   declong               660 non-null    float64
 7   decage                660 non-null    int64  
 8   smoke                 660 non-null    int64  
 9   height                660 non-null    float64
 10  weight                660 non-null    float64
 11  bmi                   660 non-null    float64
 12  health_history1       660 non-null    int64  
 13  health_history2       660 non-null    int64  
 14  health_history3       660 non-null    int64  
 15  health_history4       6